In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a admission brochure about the University, \
such as links to an About page, or a Univerity page, Admission page, academics, lpunest, research, campus life and placements.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the university, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, jobs, happenings, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [8]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a university website \
and creates a short and crisp brochure about the university for prospective in-capus admissions, online and distance education. Respond in markdown.\
Include details of university campus culture, academics, students placements, accreditations and research if you have the information."

In [9]:
def get_brochure_user_prompt(university_name, url):
    user_prompt = f"You are looking at a University called: {university_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short admission-centric brochure of the university in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:10000] # Truncate if more than 5,000 characters
    return user_prompt

In [10]:
def create_brochure(university_name,url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(university_name, url)}
          ],
    )
    result = response.choices[0].message.content
    return str(result)
#    display(Markdown(result))

In [11]:
translate_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a university brochure written in English language \
and translate it completely into the user's selected language. Respond in markdown."

In [12]:
def translate_brochure_user_prompt(university_name, url,lang):
    translate_prompt = f"You are looking at a University called: {university_name}\n"
    translate_prompt += f"Here are the contents of the University Brochure in English language; use this information and translate all into {lang} language in markdown.\n"
    translate_prompt += create_brochure(university_name,url)
    translate_prompt = translate_prompt[:10000] # Truncate if more than 5,000 characters
    return translate_prompt

In [13]:
def translate_brochure(university_name, url, lang):
    translate_response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translate_system_prompt},
            {"role": "user", "content": translate_brochure_user_prompt(university_name, url, lang)}
          ],
    )
    translate_result = translate_response.choices[0].message.content
    yield translate_result

In [14]:
import gradio as gr # oh yeah!
input_text1=gr.Textbox(label="University Name:")
input_text2=gr.Textbox(label="University URL:")
input_text3=gr.Dropdown(["Hindi", "English", "Punjabi", "Kannada", "Telugu", "Tamil"], label="Choose your Language", value="English")
view = gr.Interface(
    fn=translate_brochure,
    inputs=[input_text1,input_text2,input_text3],
    outputs=[gr.Markdown(label="Brochure:")],
    title="University Admission Brochure",
    flagging_mode="never"
)
view.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


Found links: {'links': [{'type': 'university page', 'url': 'https://www.lpu.in'}, {'type': 'admission page', 'url': 'https://admission.lpu.in/'}, {'type': 'academics page', 'url': 'https://www.lpu.in/academics/'}, {'type': 'about page', 'url': 'https://www.lpu.in/about-lpu/'}, {'type': 'placements page', 'url': 'https://www.lpu.in/placements.php'}, {'type': 'international opportunities', 'url': 'https://www.lpu.in/international/'}, {'type': 'campus life', 'url': 'https://www.lpu.in/campus-life/'}, {'type': 'programmes page', 'url': 'https://www.lpu.in/programmes/all/'}, {'type': 'scholarship page', 'url': 'https://www.lpu.in/scholarship/scholarship.php'}, {'type': 'contact page', 'url': 'https://www.lpu.in/contact-us/contact-us.php'}]}
